## Flower Classification Convolutional Neural Network Project

### Dataset

Dataset containing 4,242 images of flowers. Dataset obtained from Kaggle at https://www.kaggle.com/datasets/alxmamaev/flowers-recognition. 

Dataset contained following (class / number of images):

daisy / 764
dandelion / 1,052
rose / 784
sunflower / 733
tulip / 984

##### Preprocessing

All images are resized to 244 x 244 in this notebook.

### Findings

From performing operations on this dataset, I have found that the first convolutional neural network created to classify the flowers was not accurate. As seen by the accuracy score of around 0.60, the model is correct in its classification more often than not, but is not consistent enough to be relied upon.

## Changelog

#### Version 1

- Set root directory for image locations
- Read in images.
- Resized images to all be the same size.
- Create ImageDataGenerator.
- Split data into training and validation data.
- Created  and compiled model.
- Trained model on the data.
- Analysed accuracy of the model using accuracy and loss.

In [126]:
# Import all necessary libraries

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing import image

In [127]:
# Set root directory for class folders and target size for images

root_directory = 'flowers/'
target_size = (224, 224)

In [128]:
# Resize images in dataset to target_size

for class_folder in os.listdir(root_directory):
    class_path = os.path.join(root_directory, class_folder)
    if os.path.isdir(class_path):  # Check if it is a directory
        # Loop through all images in the class folder
        for filename in os.listdir(class_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(class_path, filename)
                img = cv2.imread(img_path)
                img_resized = cv2.resize(img, target_size)
                cv2.imwrite(img_path, img_resized)  # Overwrite the original image or save to a new file

In [129]:
# Set number of training samples used in one iteration of training
# Set number of passes through dataset

batch_size = 32
epochs = 20

In [130]:
# Create ImageDataGenerator to allow real-time data augmentation. This helps the model generalise and reduces overfitting
# Rescale pixel values from 0-255 to 0-1 to improve convergence during training
# Reserve 20% of the dataset for validation

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [131]:
# Load training data

train_generator = train_datagen.flow_from_directory(
    root_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 3457 images belonging to 5 classes.


In [132]:
# Check class indice values

print(train_generator.class_indices)

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [133]:
# Load validation data

validation_generator = train_datagen.flow_from_directory(
    root_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 860 images belonging to 5 classes.


In [134]:
# Three 2D convolution layers for RGB colours, filter size of 3x3
# Reduce height and width of feature maps to reduce number of parameters and computation in the network

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),   # flatten 3D output to 1D vector for following layers
    layers.Dense(128, activation='relu'),   # Learn high-level features by combining features learning in convolutional layers
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Same number of neurons as classes, softmax outputs probability for each class
]);

In [135]:
# Adaptive Moment Estimation optimiser because it is memory-efficient and adapts the learning rate dynamically for each parameter
# Minimise loss function during training

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [136]:
# Train model

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 30s 273ms/step - accuracy: 0.3363 - loss: 1.6618 - val_accuracy: 0.5120 - val_loss: 1.2535
Epoch 2/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5625 - loss: 1.2655 - val_accuracy: 0.4639 - val_loss: 1.3574
Epoch 3/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 171ms/step - accuracy: 0.5433 - loss: 1.1416 - val_accuracy: 0.5637 - val_loss: 1.0844
Epoch 4/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6562 - loss: 0.9676 - val_accuracy: 0.5577 - val_loss: 1.0769
Epoch 5/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 172ms/step - accuracy: 0.6261 - loss: 0.9486 - val_accuracy: 0.6262 - val_loss: 0.9597
Epoch 6/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6562 - loss: 0.8868 - val_accuracy: 0.6202 - val_loss: 0.9650
Epoch 7/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 173ms/step - accuracy: 0.7178 - loss: 0.7363 - val_accuracy: 0.6442 - val_loss: 0.9686
Epoch 8/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9062 - loss: 0.4528 - 

In [137]:
# Analyse loss and accuracy measurements

loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.6274 - loss: 2.1508
Validation loss: 2.323986530303955
Validation accuracy: 0.6000000238418579


#### Results

An accuracy value of roughly 0.60 tells us that the model is correct more often than not, but not accurate enough to be relied upon consistently. Using different batch sizes and epoch values resulted in little to no change.

In [138]:
# Test model on image of sunflower

img_path = 'flowers/sunflower/6953297_8576bf4ea3.jpg'
img = image.load_img(img_path, target_size=target_size)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
print(f'Predicted class: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted class: [3]


In [139]:
# Test model on image of daisy

img_path = 'flowers/daisy/144603918_b9de002f60_m.jpg'
img = image.load_img(img_path, target_size=target_size)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
print(f'Predicted class: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted class: [0]
